# Тестовое задание "МигКредит"

## Цель:
Построить модель логистической регрессии.

## Описание данных:

* Таблица buffer_table1

|столбец|описание|
|---|---|
|ucdb_id|уникальный id наблюдения|
|train|тип сэмпла, принимает 2 значения True и False. False – строки без разметки.|
|30+ 3MoB gt 0|целевое событие, 1 или 0, пустое там, где train – False|
|cash_hist|техническое поле, по которому нужно производить merge с остальными файлами|

* Таблица buffer_table2

|столбец|описание|
|---|---|
|Request_id|поле, к которому производится merge|
|Остальные поля – характеристики кредитной истории заемщика|

* Таблица buffer_table3

|столбец|описание|
|---|---|
|Request_id|поле, к которому производится merge.|
|Остальные поля|характеристики запросов кредитной истории заемщика.|

Baseline – > 45 gini на train и test’е

Прислать нужно csv файл с 2 столбцами, 1 – ucdb_id, 2 – predict_proba – столбец с вероятностями.

Request id это уникальный ид, по которому нужно делать merge, с исходником
create_tstamp это время создания записи
Можно считать что это дата расчета переменных
Остальные поля описывать нет смысла
Это поля - характеристики кредитной истории, что они значат не важно, важно как они влияют на целевое событие
А также таблицы 2 и 3 между собой нельзя соединить
45 джини это идеально
Можно и меньше, чтобы присылали решения с меньшими результатами)

----

## Библиотеки

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict
from sklearn.metrics import make_scorer, roc_auc_score
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from plot_metric.functions import BinaryClassification
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict

from sklearn.model_selection import GridSearchCV, cross_val_score,\
                                    KFold, train_test_split, cross_validate, ParameterGrid,\
                                    cross_validate, cross_val_predict, TimeSeriesSplit, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import confusion_matrix
# пайплайн
from sklearn.pipeline import Pipeline, make_union, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, OneHotEncoder

# дамми-регрессор
from sklearn.dummy import DummyClassifier
from itertools import combinations
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_curve
import shutil

## Функции

In [2]:
def convert_types(df):
    '''конвертирует типы (по возможности)'''
    df_c = df.copy()
    for col in df.columns:
        ser = df[col]
        try:
            ser2 = ser.astype('datetime64')
            df_c[col] = ser2
        except:
            try:
                ser_float =ser.astype(np.float32)
                ser_int =ser.astype(np.int32)
                if (ser_float == ser_int).all():
                    df_c[col] =ser_int
                else:
                    df_c[col] =ser_float
            except:
                try:
                    df_c[col] = ser.str.lower().str.strip()
                except:
                    df_c[col] = ser
    return df_c

def get_counts(ser):   
    '''
    считает доли категорий
    '''
    return ser.value_counts(normalize = True).T.add_prefix(f'{ser.name}__')    
def get_ntop(ser, n):
    '''
    выбирает топ категорий
    '''
    return ser.value_counts().head(n).index

def collect_features_v2(df_buffer_table2, idx_col):
    '''
    извлечение признаков 2
    '''
    inqAmount_5 = get_ntop(df_buffer_table2['inqAmount'], 5)
    inqAmount_10 = get_ntop(df_buffer_table2['inqAmount'], 10)
    inqAmount_20 = get_ntop(df_buffer_table2['inqAmount'], 20)
    inqAmount_50 = get_ntop(df_buffer_table2['inqAmount'], 50)
    inqAmount_100 = get_ntop(df_buffer_table2['inqAmount'], 100)
    L_inqAmount = [inqAmount_5, inqAmount_10, inqAmount_20, inqAmount_50, inqAmount_100]

    inqControlNum_5 = get_ntop(df_buffer_table2['inqControlNum'], 5)
    inqControlNum_10 = get_ntop(df_buffer_table2['inqControlNum'], 10)
    inqControlNum_20 = get_ntop(df_buffer_table2['inqControlNum'], 20)
    inqControlNum_50 = get_ntop(df_buffer_table2['inqControlNum'], 50)
    inqControlNum_100 = get_ntop(df_buffer_table2['inqControlNum'], 100)
    L_inqControlNum = [inqControlNum_5, inqControlNum_10, inqControlNum_20, inqControlNum_50, inqControlNum_100]
    
    
    idx_to_use = df_buffer_table3[idx_col].unique()
    df_features= pd.DataFrame(index= idx_to_use)
    for REQUEST_ID, subdf in tqdm_notebook(df_buffer_table3.groupby(idx_col)):

        df_features.loc[REQUEST_ID,'nrows']=subdf.shape[0]
        
        df_features.loc[REQUEST_ID, 'inqAmount__nunique'] = subdf['inqAmount'].nunique()
        df_features.loc[REQUEST_ID, 'inqControlNum__nunique'] = subdf['inqControlNum'].nunique()     
        
        row_inqAmount= pd.Series([subdf['inqAmount'].map(lambda x: x in inqAmount).mean()\
                                 for inqAmount in L_inqAmount]).to_frame().T.add_prefix('inqAmount__') 
        row_inqControlNum= pd.Series([subdf['inqControlNum'].map(lambda x: x in inqControlNum).mean()\
                                 for inqControlNum in L_inqControlNum]).to_frame().T.add_prefix('inqControlNum__')
        df_features.loc[REQUEST_ID,row_inqAmount.columns]=row_inqAmount.values
        df_features.loc[REQUEST_ID,row_inqControlNum.columns]=row_inqControlNum.values
        
        for col in ['inqPurpose', 'inquiryPeriod', 'inqPurposeText']:        
            shares = get_counts(subdf[col]).to_frame(REQUEST_ID).T
            df_features.loc[REQUEST_ID,shares.columns]=shares.values
            
    return df_features


def collect_features_v1(df_buffer_table2, idx_col):
    '''
    извлечение признаков 1
    '''
    paymtPat_5 = get_ntop(df_buffer_table2['paymtPat'], 5)
    paymtPat_10 = get_ntop(df_buffer_table2['paymtPat'], 10)
    paymtPat_20 = get_ntop(df_buffer_table2['paymtPat'], 20)
    paymtPat_50 = get_ntop(df_buffer_table2['paymtPat'], 50)
    paymtPat_100 = get_ntop(df_buffer_table2['paymtPat'], 100)
    L_paymtPat = [paymtPat_5, paymtPat_10, paymtPat_20, paymtPat_50, paymtPat_100]

    monthsReviewed_5 = get_ntop(df_buffer_table2['monthsReviewed'], 5)
    monthsReviewed_10 = get_ntop(df_buffer_table2['monthsReviewed'], 10)
    monthsReviewed_20 = get_ntop(df_buffer_table2['monthsReviewed'], 20)
    monthsReviewed_50 = get_ntop(df_buffer_table2['monthsReviewed'], 50)
    monthsReviewed_100 = get_ntop(df_buffer_table2['monthsReviewed'], 100)
    L_monthsReviewed = [monthsReviewed_5, monthsReviewed_10, monthsReviewed_20, monthsReviewed_50, monthsReviewed_100]
    
    idx_to_use = df_buffer_table2[idx_col].unique()
    df_features= pd.DataFrame(index= idx_to_use)
    for REQUEST_ID, subdf in tqdm_notebook(df_buffer_table2.groupby(idx_col)):

        df_features.loc[REQUEST_ID,'nrows']=subdf.shape[0]

        row_paymtPat= pd.Series([subdf['paymtPat'].map(lambda x: x in paymtPat).mean()\
                                 for paymtPat in L_paymtPat]).to_frame().T.add_prefix('paymtPat__') 
        row_monthsReviewed= pd.Series([subdf['monthsReviewed'].map(lambda x: x in monthsReviewed).mean()\
                                 for monthsReviewed in L_monthsReviewed]).to_frame().T.add_prefix('monthsReviewed__')
        df_features.loc[REQUEST_ID,row_paymtPat.columns]=row_paymtPat.values
        df_features.loc[REQUEST_ID,row_monthsReviewed.columns]=row_monthsReviewed.values

        df_features.loc[REQUEST_ID,'UUID_nanshare']= subdf['UUID'].isna().mean()

        for col in ['currencyCode', 'paymtFreqText', 'interestPaymentFrequencyText',
                    'accountRatingText', 'acctTypeText', 'BUSINESSCATEGORY',\
                    'accountRating', 'acctType', 'numDays60', 'numDays30', 'numDays90',
                    'termsFrequency', 'interestPaymentFrequencyCode']:        
            shares = get_counts(subdf[col]).to_frame(REQUEST_ID).T
            df_features.loc[REQUEST_ID,shares.columns]=shares.values

        for col in ['amtPastDue', 'curBalanceAmt', 'termsAmt', 'amtOutstanding', 'CREDITTOTALAMT', 'PRINCIPALOUTSTANDING',
                    'PRINCIPALPASTDUE', 'OTHERAMTOUTSTANDING', 'OTHERAMTPASTDUE',
                    'INTOUTSTANDING', 'INTPASTDUE', 'creditLimit', 'numDays60', 'numDays30', 'numDays90']:
            df_features.loc[REQUEST_ID, '{}__sum'.format(str(col))] = subdf[col].sum()
            df_features.loc[REQUEST_ID, '{}__mean'.format(str(col))] = subdf[col].mean()
            df_features.loc[REQUEST_ID, '{}__max'.format(str(col))] = subdf[col].max()
            df_features.loc[REQUEST_ID, '{}__min'.format(str(col))] = subdf[col].min()

        for col in  ['openedDt', 'lastPaymtDt', 'closedDt', 'reportingDt', 'paymtPatStartDt',
                     'lastUpdatedDt', 'accountRatingDate', 'paymentDueDate',
                     'interestPaymentDueDate', 'DATE_RESPONSE', 'CREATE_TSTAMP',
                     'ETL_TIMESTAMP', 'GRACEENDDT']:
            year_shares = subdf[col].dt.year.value_counts(normalize= True)
            month_shares = subdf[col].dt.month.value_counts(normalize= True)
            day_shares = subdf[col].dt.day.value_counts(normalize= True)
            dow_shares = subdf[col].dt.dayofweek.value_counts(normalize= True)
            hour_shares = subdf[col].dt.round('H').dt.hour.value_counts(normalize= True)
            for name, ser in zip(('year_shares', 'month_shares', 'day_shares', 'dow_shares', 'hour_shares'),\
                                 (year_shares, month_shares, day_shares, dow_shares, hour_shares)):
                for k, v in ser.items():
                    df_features.loc[REQUEST_ID, f'{name}__{k}'] = v    

            lifetime = (subdf[col].max() - subdf[col].min())/np.timedelta64(1, 's')
            df_features.loc[REQUEST_ID, 'lifetime'] = lifetime 
            
    return df_features


class SklearnHelperTargetEncoder(BaseEstimator, TransformerMixin):
    ''' Кодирование категорий с помощью целевой переменной '''
    def __init__(self, n_iter, n_folds, min_samples_leaf, seed):
        self.n_iter = n_iter
        self.n_folds = n_folds
        self.min_samples_leaf = min_samples_leaf
        self.seed = seed
    def fit(self, X, y=None):
        self.y_mean = y.mean()
        _df_tr = pd.concat([X, y], 1)
        target_col = _df_tr.columns[-1]
        to_encode = _df_tr.columns[:-1]
        
        L_tr = []        
        self.L_d_encs = []
        for i in tqdm_notebook(range(self.n_iter)): 
            enc_tr = pd.DataFrame(index = _df_tr.index, columns = to_encode).fillna(0.0)
            for col in to_encode:
                for tr_idx, val_idx in KFold(self.n_folds, shuffle = True,random_state = self.seed+i)\
                                       .split(_df_tr):                    
                    grp = _df_tr.iloc[tr_idx].groupby(col)[target_col].agg({'mean', 'count'}) 
                    d_enc = grp[grp['count']>=self.min_samples_leaf]['mean'].to_dict()
                    self.L_d_encs.append((col, d_enc))
                    to_enc_tr =_df_tr.iloc[val_idx]                    
                    enc_tr.loc[to_enc_tr.index, col] = to_enc_tr[col].map(d_enc)                  
            L_tr.append(enc_tr)    
            
        self.enc_tr =  pd.concat(L_tr, 1)
        self._df_tr = _df_tr
        return self    
    def transform(self, X):
        if np.all(X.values == self._df_tr.values):
            return self.enc_tr.fillna(self.y_mean) 
        else:
            df_enc = pd.DataFrame(index = X.index, columns=X.columns).fillna(0.0)
            for feat, d in tqdm_notebook(self.L_d_encs):
                df_enc.loc[:, feat] += X[feat].map(d) / self.n_iter
            return df_enc.fillna(self.y_mean)        

class SklearnHelperFeatureSelector(BaseEstimator, TransformerMixin):
    ''' Отбор признаков '''
    def __init__(self, model, cv, scoring, show_progress):
        self.model = model
        self.cv = cv
        self.scoring = scoring
        self.show_progress = show_progress
    def fit(self, X, y=None):
        #assert (isinstance(X, np.ndarray)) or (X.getformat() == 'csc')
        _X = X.copy()
        cv_scores = []
        for i in tqdm_notebook(range(_X.shape[1])):
            try:
                _X_curr = _X[:, i].toarray().reshape(-1,1)
            except:
                _X_curr = _X[:, i].reshape(-1,1)                
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()            
            cv_scores.append(mean_cv_score)
            
        order = np.argsort(cv_scores)[::-1]
        to_drop_before, best_features, best_cv_score = [], [order[0]], -np.inf
        for i in tqdm_notebook(order[1:]):
            curr_features = best_features+[i]
            _X_curr = _X[:, curr_features]
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
            if mean_cv_score>best_cv_score:
                best_cv_score = mean_cv_score
                best_features = curr_features
                if self.show_progress:
                    print('new best score = {:.10f}'.format(best_cv_score))
            else:
                to_drop_before.append(i)
        while True:
            to_drop_after = []
            for i in tqdm_notebook(to_drop_before):
                curr_features = best_features+[i]
                _X_curr = _X[:, curr_features]
                mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
                if mean_cv_score>best_cv_score:
                    best_cv_score = mean_cv_score
                    best_features = curr_features
                    if self.show_progress:
                        print('new best score = {:.10f}'.format(best_cv_score))
                else:
                    to_drop_after.append(i)
            if to_drop_before == to_drop_after:
                break
            else:
                to_drop_before = to_drop_after  
        self.best_features_ = best_features
        self.best_score_ = best_cv_score
    def transform(self, _X):
        return _X[:, self.best_features_]
    
def gini(y_trues, y_predprob):
    return (2 * roc_auc(y_trues, y_predprob[:, 1])) - 1
gini_scorer = make_scorer(gini, larger_is_better=True)

## Решение
* признаки извлек в другой тетрадке с помощью функций: _collect_features_v1_, _collect_features_v2_ 
* извлекались счетчики, уникальные счетчики, суммы,средние, минимум, максимум

In [3]:
with open('X_tr', 'rb') as f:
    X_tr=pickle.load(f)    
with open('y_tr', 'rb') as f:
    y_tr=pickle.load(f)
with open('X_te', 'rb') as f:
    X_te=pickle.load(f)
with open('y_te', 'rb') as f:
    y_te=pickle.load(f)

assert (X_tr.index== y_tr.index).all()
assert (X_te.index== y_te.index).all()

In [14]:
SEED = 13
NFOLDS = 5

In [5]:
cols_to_use = list(set(X_tr.columns).intersection(set(X_te.columns)))
X_tr = X_tr[cols_to_use]
X_te = X_te[cols_to_use]

nuniques = X_tr.nunique()
X_tr = X_tr[nuniques[nuniques>1].index]
X_te = X_te[nuniques[nuniques>1].index]

X_tr.columns = range(len(X_tr.columns))
X_te.columns = range(len(X_te.columns))

In [8]:
nuniques_v2 = X_tr.nunique()
to_enc = nuniques_v2[nuniques_v2<=100].index.tolist()
encoder = SklearnHelperTargetEncoder(n_iter=20, n_folds=20, min_samples_leaf=5, seed=SEED)

encoder.fit(X_tr[to_enc], y_tr)
X_enc_tr = encoder.transform(X_tr[to_enc])
X_enc_te = encoder.transform(X_te[to_enc])

SklearnHelperTargetEncoder(min_samples_leaf=5, n_folds=20, n_iter=20, seed=13)

In [11]:
X2_tr = np.column_stack([X_tr.values, X_enc_tr.values])
X2_te = np.column_stack([X_te.values, X_enc_te.values])

In [12]:
scaler = MinMaxScaler()
scaler.fit(X2_tr)
_X2_tr = scaler.transform(X2_tr)
_X2_te = scaler.transform(X2_te)
_y_tr = y_tr.copy()
_y_te = y_te.copy()
_X2_tr[np.isnan(_X2_tr)] = -1
_X2_te[np.isnan(_X2_te)] = -1 

In [15]:
# валидация, модель
SKF = StratifiedKFold(NFOLDS, random_state = SEED, shuffle = True)    
logit= LogisticRegression(random_state = SEED) 

In [16]:
# отбираем признаки
selector = SklearnHelperFeatureSelector(model=logit,\
                                        cv=SKF,
                                        scoring='roc_auc',\
                                        show_progress = True)
selector.fit(_X2_tr, _y_tr)

new best score = 0.6340854176
new best score = 0.6743846075
new best score = 0.6940894347
new best score = 0.7090809617
new best score = 0.7181546969
new best score = 0.7187592583
new best score = 0.7236650335
new best score = 0.7274228682
new best score = 0.7371757232
new best score = 0.7372691819
new best score = 0.7376311009
new best score = 0.7401619974
new best score = 0.7402866981
new best score = 0.7413180488
new best score = 0.7416612761
new best score = 0.7442770198
new best score = 0.7467665774
new best score = 0.7472056667
new best score = 0.7541206800
new best score = 0.7543168265
new best score = 0.7547732724
new best score = 0.7551857092
new best score = 0.7573354108
new best score = 0.7576994660
new best score = 0.7577424737
new best score = 0.7577498670
new best score = 0.7587931504
new best score = 0.7615546897
new best score = 0.7616446605
new best score = 0.7627206211
new best score = 0.7632295040
new best score = 0.7633362973
new best score = 0.7660826663
new best s

new best score = 0.7996489622
new best score = 0.7996606613
new best score = 0.8004160250
new best score = 0.8004928614
new best score = 0.8007614552
new best score = 0.8008178642
new best score = 0.8010599391
new best score = 0.8011332708
new best score = 0.8011659480
new best score = 0.8011856745
new best score = 0.8016476278
new best score = 0.8016922376
new best score = 0.8017741142
new best score = 0.8018078428
new best score = 0.8018778033
new best score = 0.8018814082
new best score = 0.8023095494



In [17]:
x_sel_tr = selector.transform(_X2_tr)
x_sel_te = selector.transform(_X2_te)

In [18]:
gs = GridSearchCV(logit, param_grid = {'C':np.logspace(-3,3,100)}, scoring = 'roc_auc', cv =SKF)
gs.fit(x_sel_tr, _y_tr)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=LogisticRegression(random_state=13),
             param_grid={'C': array([1.00000000e-03, 1.14975700e-03, 1.32194115e-03, 1.51991108e-03,
       1.74752840e-03, 2.00923300e-03, 2.31012970e-03, 2.65608778e-03,
       3.05385551e-03, 3.51119173e-03, 4.03701726e-03, 4.64158883e-03,
       5.33669923e-03, 6.13590727e-0...
       4.03701726e+01, 4.64158883e+01, 5.33669923e+01, 6.13590727e+01,
       7.05480231e+01, 8.11130831e+01, 9.32603347e+01, 1.07226722e+02,
       1.23284674e+02, 1.41747416e+02, 1.62975083e+02, 1.87381742e+02,
       2.15443469e+02, 2.47707636e+02, 2.84803587e+02, 3.27454916e+02,
       3.76493581e+02, 4.32876128e+02, 4.97702356e+02, 5.72236766e+02,
       6.57933225e+02, 7.56463328e+02, 8.69749003e+02, 1.00000000e+03])},
             scoring='roc_auc')

In [23]:
y_predprob_cv = cross_val_predict(gs.best_estimator_, x_sel_tr, _y_tr, cv = SKF, method='predict_proba')[:,1]
y_predprob_te = gs.best_estimator_.predict_proba(x_sel_te)[:,1]

In [29]:
GINI_cv = 100*((2 * roc_auc_score(_y_tr, y_predprob_cv)) - 1)

In [46]:
GINI_cv

60.49474767832157

* Baseline – > 45 gini на train и test’е
* Прислать нужно csv файл с 2 столбцами, 1 – ucdb_id, 2 – predict_proba – столбец с вероятностями.

In [50]:
df_predprob_cv = pd.Series(y_predprob_cv).to_frame('predict_proba')
df_predprob_cv['ucdb_id'] = np.int64(y_tr.index)

In [51]:
df_predprob_te = pd.Series(y_predprob_te).to_frame('predict_proba')
df_predprob_te['ucdb_id'] = np.int64(y_te.index)

In [52]:
df_predprob_te.to_excel('predict_proba_TE.xlsx')
df_predprob_cv.to_excel('predict_proba_CV(gini=60.5).xlsx')

In [47]:
y_te.index

Int64Index([1068809235, 1069323927, 1070123601, 1069249312, 1062806411,
            1064180804, 1065696254, 1062459772, 1069975500, 1070609484,
            ...
            1068285451, 1068693464, 1068621594, 1068359972, 1066715545,
            1062391323, 1062331468, 1068496544, 1071147256, 1066893283],
           dtype='int64', name='ucdb_id', length=2414)

In [49]:
df_predprob_te

,predict_proba,ucdb_id
0,0.076383,1068809235
1,0.016906,1069323927
2,0.087695,1070123601
3,0.076547,1069249312
4,0.277512,1062806411
...,...,...
2409,0.035537,1062391323
2410,0.433791,1062331468
2411,0.023584,1068496544
2412,0.068864,1071147256
